<center>

**Vježba 1 - MLServer**

</center>

U ovoj vježbi želimo vidjeti kako pokrenuti modele na ML serveru lokalno na računalu.
Prvo instaliramo potrebne module.

In [ ]:
%%bash
pip install --upgrade pip
pip install mlserver mlserver-mlflow mlserver-sklearn

In [ ]:
%%bash
mkdir mlserver_iris

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from joblib import dump, load

Učitamo iris skup podataka. Odradimo standardno skaliranje i učenje MLP-a.  
Spremimo MLP u joblib datoteku.

In [ ]:
def loadIrisDataset():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    return (X,y)

(X,y)=loadIrisDataset()

scaler = StandardScaler()

X_scal = scaler.fit_transform(X)
X_scal = pd.DataFrame(X_scal)

X_train, X_test, y_train, y_test = train_test_split(X_scal, y, test_size = 0.2)

mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=2000, verbose=1)
mlp.fit(X_train, y_train)

dump(mlp, './mlserver_iris/mlp_iris.joblib')

Definiramo opisnike za MLServer.  
Pokažemo na naš MLP model u joblib datoteci.

In [ ]:
%%writefile ./mlserver_iris/settings.json
{
    "debug": "true"
}

In [ ]:
%%writefile ./mlserver_iris/model-settings.json
{
    "name": "mlp-iris",
    "implementation": "mlserver_sklearn.SKLearnModel",
    "parameters": {
        "uri": "./mlp_iris.joblib",
        "version": "v1.0.0"
    }
}

Postavimo se u direktorij s modelom i opisnicima, te startamo MLServer.

In [ ]:
%%bash
cd mlserver_iris
mlserver start . &

Poziv REST endpoint-a. V2 *inreference* protokol.

In [ ]:
import requests

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": (2,4),
          "datatype": "FP32",
          "data": [[-1.143017, 1.249201, -1.340227, -1.447076],[0.432165, 0.788808, 0.933271, 1.448832]]
        }
    ]
}

endpoint = "http://localhost:8080/v2/models/mlp-iris/versions/v1.0.0/infer"
response = requests.post(endpoint, json=inference_request)

response.json()

Pošaljemo podatke iz testnog skupa podataka.

In [ ]:
inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": X_test.shape,
          "datatype": "FP32",
          "data": X_test.values.tolist()
        }
    ]
}

endpoint = "http://localhost:8080/v2/models/mlp-iris/versions/v1.0.0/infer"
response = requests.post(endpoint, json=inference_request)

response.json()

Izgeneriramo custom server. Ovo se može deployat na SC2.

In [ ]:
%%bash
cd mlserver_iris
mlserver build . -t 'my-super-iris-server:1.0.0'

In [ ]:
%%bash
docker run -d --net=host my-super-iris-server:1.0.0

In [ ]:
%%bash
docker rm $(docker stop $(docker ps -a -q --filter ancestor=my-super-iris-server:1.0.0 --format="{{.ID}}"))